# 📘 Gemma 2B – Merge LoRA

- **Author:** Ederson Corbari <e@NeuroQuest.ai>
- **Date:** February 01, 2026  

---

## Overview

This notebook demonstrates an end-to-end workflow for loading, merging, and running inference with the **Gemma 2B Instruction-Tuned Large Language Model (LLM)** augmented with a **LoRA adapter trained on a psychological preference dataset**.

The notebook focuses on:
- Loading the base Gemma 2B instruction model
- Attaching and merging a LoRA adapter for behavioral alignment
- Exporting and publishing the merged model
- Running a lightweight inference sanity check

Rather than performing training inside the notebook, this workflow validates the **model integration, merging process, and inference behavior**, ensuring the resulting model is ready for downstream use in psychologically safe and empathetic conversational settings.

The final merged model is publicly available on the Hugging Face Hub at:

- **https://huggingface.co/ecorbari/Gemma-2b-it-Psych-Merged**

---


## 1️⃣ Introduction

This notebook provides a minimal and practical pipeline for **merging a LoRA-adapted Gemma 2B model and validating it through inference**.

The LoRA adapter used in this workflow was trained separately using preference-based data designed to encourage psychologically safe, empathetic, and therapeutically aligned responses. Here, the focus is on correctly loading the base model, applying the adapter, merging the weights, and verifying that the final model behaves as expected during inference.

The primary objectives of this notebook are to:

- Load the Gemma 2B instruction-tuned base model efficiently
- Attach and merge a LoRA adapter into the base model
- Export and publish the merged model for reuse
- Perform a lightweight inference test to validate behavior and environment setup

This notebook is intentionally lightweight and serves as a **validation and deployment step**, rather than a full training pipeline. It is well-suited for rapid iteration, sanity checks, and preparation of aligned models for downstream applications.


## 2️⃣ Environment & Dependencies

This notebook assumes:
- PyTorch with CUDA support
- Hugging Face Transformers
- bitsandbytes (for 4-bit quantization)

In [1]:
import warnings
warnings.simplefilter("ignore")

In [2]:
from dotenv import load_dotenv
from pathlib import Path

env_path = Path("../.env")
load_dotenv(dotenv_path=env_path)

True

In [3]:
import torch

from typing import Final
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
from peft import PeftModel

In [49]:
assert torch.cuda.is_available(), "GPU CUDA not found"
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_capability(0))

NVIDIA T1000 8GB
(7, 5)


In [6]:
BASE_MODEL: Final[str] = "google/gemma-2b-it"
LORA_MODEL: Final[str] = "ecorbari/Gemma-2b-it-Psych"  
MERGED_MODEL: Final[str] = "Gemma-2b-it-Psych-Merged"

## 3️⃣ Load Base Model

Loads a causal LLM with automatic device placement and reduced memory usage.

In [36]:
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    dtype=torch.float16,
    device_map="auto",
)

Loading weights:   0%|          | 0/164 [00:00<?, ?it/s]

## 4️⃣ Load Tokenizer

Loads the tokenizer and uses the EOS token as the padding token.

In [37]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

## 5️⃣ Load and Merge LoRA Adapter

Loads the LoRA fine-tuned adapters on top of the base model.

In [38]:
model = PeftModel.from_pretrained(
    base_model,
    LORA_MODEL,
)

model = model.merge_and_unload()

In [39]:
model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): GemmaRMSNorm((2048,), 

## 6️⃣ Save Merged Model

Saves the merged model weights and tokenizer to disk.

In [40]:
model.save_pretrained(MERGED_MODEL)
tokenizer.save_pretrained(MERGED_MODEL)

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('Gemma-2b-it-Psych-Merged/tokenizer_config.json',
 'Gemma-2b-it-Psych-Merged/chat_template.jinja',
 'Gemma-2b-it-Psych-Merged/tokenizer.json')

## 7️⃣ Saving and Exporting the Model

Uploads the merged model and tokenizer to the [Hugging Face Hub](https://huggingface.co/ecorbari/Gemma-2b-it-Psych-Merged).

In [41]:
model.push_to_hub(MERGED_MODEL)
tokenizer.push_to_hub(MERGED_MODEL)

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/ecorbari/Gemma-2b-it-Psych-Merged/commit/5e3daaf83c75a5234d00e59f7c310007711883b5', commit_message='Upload tokenizer', commit_description='', oid='5e3daaf83c75a5234d00e59f7c310007711883b5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ecorbari/Gemma-2b-it-Psych-Merged', endpoint='https://huggingface.co', repo_type='model', repo_id='ecorbari/Gemma-2b-it-Psych-Merged'), pr_revision=None, pr_num=None)

## 8️⃣ Inference Test

Sets up a text-generation inference pipeline.

In [4]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.memory_allocated() / 1024**2, "MB")
print(torch.cuda.memory_reserved() / 1024**2, "MB")

True
NVIDIA T1000 8GB
0.0 MB
0.0 MB


In [7]:
pipe = pipeline(
    "text-generation",
    model=MERGED_MODEL,
    dtype=torch.float16,
    device_map="auto",
)

Loading weights:   0%|          | 0/164 [00:00<?, ?it/s]

In [10]:
from IPython.display import display, Markdown

prompt = "I feel anxious and overwhelmed lately. What should I do?"
result = pipe(prompt)

response = result[0]["generated_text"][len(prompt) :].strip()

display(
    Markdown(f"""
### 🤖 Model Response

{response}
""")
)

Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



### 🤖 Model Response

model
It's understandable to feel anxious and overwhelmed, especially in uncertain times. One strategy that could help is deep breathing exercises. Take slow, deep breaths and focus on your breath. You can also try progressive muscle relaxation, which involves gradually tensing and relaxing different muscle groups in your body. Additionally, it may be helpful to identify the triggers that cause your anxiety and work on developing coping strategies to manage them. Remember to be kind and compassionate with yourself, and seek support from loved ones or a therapist if necessary.
